In [0]:
#@title Install additional packages and mount Google Drive { vertical-output: true }
from IPython.display import clear_output
import os
os.system('apt install mediainfo')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
clear_output(wait=True)
print('Installation finished. Now mount your Google Drive.')
from google.colab import drive
drive.mount('/content/drive')
clear_output(wait=True)
print('Drive is mounted. Setup is completed.')

Drive is mounted. Setup is completed.


# <img src='https://i.imgur.com/N4bdPff.png' height="30" alt="IMDB"/> **Generate IMDB template**

In [0]:
import urllib.request
import json
import re
import requests
import sys
import subprocess
import os
import time
from IPython.display import clear_output
from bs4 import BeautifulSoup

# Variables
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.8; rv:21.0) Gecko/20100101 Firefox/21.0'}
api_key = '991109a2'
#@markdown ###IMDB Informations
IMDB_URL_OR_ID = 'tt0092099' #@param {type:"string"}
imdb_pattern = re.compile('(tt\d{7,8})')
youtube_pattern = re.compile('(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})')
imdb_id = re.findall(imdb_pattern, IMDB_URL_OR_ID)[0]

def get_imdb_template(imdb_id):
  imdb_url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}&r=json&plot=full'
  Path = "/content/drive/Shared drives/TDN Share Drive/Movies/Top.Gun.1986.SE.1080p.BluRay.x264-CLASSiC/tg-cls.mkv" #@param {type:"string"}
  Link = "https://drive.google.com/open?id=19wjHlDnTfQ1hhJyQtMc7f9VuvQw3ZzUz" #@param {type:"string"}
  Screenshot = "" #@param {type:"string"}
  Trailer = "" #@param {type:"string"}
  youtube_match = re.match(youtube_pattern, Trailer)
  if Trailer != "":
    if not youtube_match:
      clear_output()
      print('Your trailer link is invalid. Please only put in youtube link!')
      sys.exit()
    else:
      trailer_enabled = True
  else:
    trailer_enabled = False

  if Screenshot != "":
    screenshot_list = Screenshot.split(" ")
    screenshot_enabled = True
  else:
    screenshot_enabled = False

  if Path == "":
    mediainfo_enabled = False
  else:
    mediainfo_enabled = True
    tmp = subprocess.Popen('mediainfo --Logfile=/root/.nfo "{}"'.format(Path), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    time.sleep(5) # Wait 5s for the nfo file to populate properly
    with open('/root/.nfo', 'r', encoding='utf-8') as nfo_file:
      nfo = nfo_file.readlines()
      del nfo[1]
      nfo[1] = f"Title name                               : {os.path.splitext(os.path.basename(Path))[0].replace('.', ' ')} [{nfo[4][43:].strip()}] [{nfo[5][43:].strip()}]\n"
      nfo = ''.join(nfo)

  if Link == "":
    print("You forgot putting in the link to download!")
    sys.exit()

  #@markdown ---
  #@markdown ###Forum Requirements
  Likes = 2 #@param {type: "number"}
  Posts = 2 #@param {type: "number"}
  Thanks = True #@param {type:"boolean"}

  def get_story_line(imdb_id):
    url = 'https://www.imdb.com/title/{}'.format(imdb_id)
    r = requests.get(url, headers=headers)
    data = r.text
    soup = BeautifulSoup(data, 'html.parser')
    story_line = soup.find_all('div', class_='inline canwrap')[0].find('span').text.lstrip()
    return story_line

  # Fetch data
  with urllib.request.urlopen(imdb_url) as imdb_url:
    imdb_data = json.loads(imdb_url.read().decode())

  # Process data
  text_dump = "[center][img]{}[/img]\n".format(imdb_data['Poster'])
  text_dump += "[color=rgb(250, 197, 28)][b][size=6]{} ({})[/size][/b][/color]\n".format(imdb_data['Title'], imdb_data['Year'])
  text_dump += "[url=https://www.imdb.com/title/{}][img]https://i.imgur.com/rcSipDw.png[/img][/url][size=6][b] {}[/b]/10[/size]".format(imdb_id, imdb_data['imdbRating'])
  text_dump += "[size=6] --- [img]https://i.imgur.com/sEpKj3O.png[/img]{}[/size][/center]\n".format(imdb_data['imdbVotes'])
  text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Plot Summary[/b][/color][/size][/indent]\n"
  text_dump += "{}\n".format(get_story_line(imdb_id))
  if trailer_enabled:
    text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Trailer[/b][/color][/size][/indent]\n"
    text_dump += "[media=youtube]{}[/media]\n".format(youtube_match.group(6))
  if screenshot_enabled:
    text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Screenshot[/b][/color][/size][/indent]\n"
    text_dump += "[spoiler='Click here to view screenshots']\n"
    for i in screenshot_list:
      text_dump += "[img]{}[/img]\n".format(i)
    text_dump += "[/spoiler]"
  text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Movie Info[/b][/color][/size][/indent]\n"
  text_dump += "[LIST][*][color=rgb(255, 204, 0)]Rating:[/color] {}\n".format(imdb_data['Rated'])
  text_dump += "[*][color=rgb(255, 204, 0)]Genre:[/color] {}\n".format(imdb_data['Genre'])
  text_dump += "[*][color=rgb(255, 204, 0)]Directed By:[/color] {}\n".format(imdb_data['Director'])
  text_dump += "[*][color=rgb(255, 204, 0)]Written By:[/color] {}\n".format(imdb_data['Writer'])
  text_dump += "[*][color=rgb(255, 204, 0)]Starring:[/color] {}\n".format(imdb_data['Actors'])
  text_dump += "[*][color=rgb(255, 204, 0)]Release Date:[/color] {}\n".format(imdb_data['Released'])
  try:
    text_dump += "[*][color=rgb(255, 204, 0)]Runtime:[/color] {}\n".format(imdb_data['Runtime'])
  except:
    text_dump += "[*][color=rgb(255, 204, 0)]Runtime:[/color] N/A\n"
  try:
    text_dump += "[*][color=rgb(255, 204, 0)]On Disk/Streaming:[/color] {}\n".format(imdb_data['DVD'])
  except:
    text_dump += "[*][color=rgb(255, 204, 0)]On Disk/Streaming:[/color] N/A\n"
  try:
    text_dump += "[*][color=rgb(255, 204, 0)]Box Office:[/color] {}\n".format(imdb_data['BoxOffice'])
  except:
    text_dump += "[*][color=rgb(255, 204, 0)]Box Office:[/color] N/A\n"
  text_dump += "[*][color=rgb(255, 204, 0)]Awards:[/color] {}\n".format(imdb_data['Awards'])
  try:
    text_dump += "[*][color=rgb(255, 204, 0)]Studio:[/color] {}[/LIST]\n".format(imdb_data['Production'])
  except:
    text_dump += "[*][color=rgb(255, 204, 0)]Studio:[/color] N/A[/LIST]\n"
  if mediainfo_enabled:
    text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Media Info[/b][/color][/size][/indent]\n"
    text_dump += "[spoiler='Click here to view Media Info'][code]{}[/code][/spoiler]\n".format(nfo)
  text_dump += "[hr][/hr][center][size=6][color=rgb(250, 197, 28)][b]Download Link[/b][/color][/size][/center]\n"
  if Thanks:
    if Likes != 0 and Posts == 0:
      text_dump += "[center][thanks][likes={}]{}[/likes][/thanks][/center]".format(Likes, Link)
    elif Likes == 0 and Posts != 0:
      text_dump += "[center][thanks][posts={}]{}[/posts][/thanks][/center]".format(Posts, Link)
    elif Likes != 0 and Posts != 0:
      text_dump += "[center][thanks][likes={}][posts={}]{}[/posts][/likes][/thanks][/center]".format(Likes, Posts, Link)
    else:
      text_dump += "[center][thanks]{}[/thanks][/center]".format(Link)
  else:
    if Likes != 0 and Posts == 0:
      text_dump += "[center][likes={}]{}[/likes][/center]".format(Likes, Link)
    elif Likes == 0 and Posts != 0:
      text_dump += "[center][posts={}]{}[/posts][/center]".format(Posts, Link)
    elif Likes != 0 and Posts != 0:
      text_dump += "[center][likes={}][posts={}]{}[/posts][/likes][/center]".format(Likes, Posts, Link)
    else:
      text_dump += "[center]{}[/center]".format(Link)

  return text_dump

print(get_imdb_template(imdb_id))

[center][img]https://m.media-amazon.com/images/M/MV5BZjQxYTA3ODItNzgxMy00N2Y2LWJlZGMtMTRlM2JkZjI1ZDhhXkEyXkFqcGdeQXVyNDk3NzU2MTQ@._V1_SX300.jpg[/img]
[color=rgb(250, 197, 28)][b][size=6]Top Gun (1986)[/size][/b][/color]
[url=https://www.imdb.com/title/tt0092099][img]https://i.imgur.com/rcSipDw.png[/img][/url][size=6][b] 6.9[/b]/10[/size][size=6] --- [img]https://i.imgur.com/sEpKj3O.png[/img]267,072[/size][/center]
[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Plot Summary[/b][/color][/size][/indent]
Code-named "Maverick", Pete Mitchell, the impetuous daredevil Navy-pilot ace, is accepted into Miramar's elite Fighter School, also known as "Top Gun". There, as the impulsive pilot competes with the best of the best, not only will he meet Charlie, the flying school's curvaceous astrophysics instructor, but also the brilliant and highly competitive fellow student, "Iceman", with whom right from the start, he will engage in a reckless contest. As Maverick is haunted by his father's my

# <img src='https://i.imgur.com/e5Thsob.jpg?1' height="25" alt="MAL"/> **Generate MAL template**

In [0]:
import re
import sys
import os
import time
import subprocess
from bs4 import BeautifulSoup
from IPython.display import clear_output
from jikanpy import Jikan

jikan = Jikan()

#@markdown ###MAL Information
Anime_URL_OR_ID = '' #@param {type: "string"}
mal_pattern = re.compile('(anime)?/(\d{1,7})/(.+)?')
if re.match(r'\d+', Anime_URL_OR_ID):
  mal_id = Anime_URL_OR_ID
else:
  mal_id = re.findall(mal_pattern, Anime_URL_OR_ID)[0][1]
youtube_pattern = re.compile('(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})')

def get_mal_template(mal_id):
  # Fetch data
  anime_data = jikan.anime(mal_id)
  #@markdown ---
  #@markdown ###Anime Information
  Screenshot = "" #@param {type:"string"}
  if Screenshot != "":
    screenshot_list = Screenshot.split(" ")
    screenshot_enabled = True
  else:
    screenshot_enabled = False

  def get_info(anime_data, info):
    text = ''
    for i in range(len(anime_data[f'{info}'])):
      text += anime_data[info][i]['name'] + ', '
    return text[:-2]

  Path = "" #@param {type:"string"}
  if Path == "":
    mediainfo_enabled = False
  else:
    mediainfo_enabled = True
    tmp = subprocess.Popen('mediainfo --Logfile=/root/.nfo "{}"'.format(Path), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    time.sleep(5) # Wait 5s for the nfo file to populate properly
    with open('/root/.nfo', 'r', encoding='utf-8') as nfo_file:
      nfo = nfo_file.readlines()
      del nfo[1]
      nfo[1] = "Complete name                            : {}\n".format(os.path.basename(Path))
      nfo = ''.join(nfo)

  Link = "" #@param {type:"string"}
  if Link == "":
    print("You forgot putting in the link to download!")
    sys.exit()

  #@markdown ---
  #@markdown ###Forum Requirements
  Likes = 0 #@param {type: "number"}
  Posts = 0 #@param {type: "number"}
  Thanks = True #@param {type:"boolean"}

  # Process data
  text_dump = "[center][img]{}[/img]\n".format(anime_data['image_url'])
  text_dump += "[color=rgb(250, 197, 28)][b][size=6]{} ({} - {})[/size][/b][/color]\n".format(anime_data['title'], anime_data['type'], anime_data['premiered'])
  text_dump += "[url={}][img]https://i.imgur.com/YJgzM4V.png[/img][/url][size=6][b] {}[/b]/10[/size]".format(anime_data['url'], anime_data['score'])
  text_dump += "[size=6] --- [img]https://i.imgur.com/sEpKj3O.png[/img]{}[/size][/center]\n".format(anime_data['scored_by'])
  text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Synopsis[/b][/color][/size][/indent]\n"
  text_dump += "{}\n".format(anime_data['synopsis'])
  try:
    text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Trailer[/b][/color][/size][/indent]\n"
    text_dump += "[media=youtube]{}[/media]\n".format(re.match(youtube_pattern, str(anime_data['trailer_url'])).group(6))
  except:
    pass
  if screenshot_enabled:
    text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Screenshot[/b][/color][/size][/indent]\n"
    text_dump += "[spoiler='Click here to view screenshots']\n"
    for i in screenshot_list:
      text_dump += "[img]{}[/img]\n".format(i)
    text_dump += "[/spoiler]"
  text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Anime Info[/b][/color][/size][/indent]\n"
  text_dump += "[LIST][*][color=rgb(255, 204, 0)]Rating:[/color] {}\n".format(anime_data['rating'])
  text_dump += "[*][color=rgb(255, 204, 0)]Type:[/color] {}\n".format(anime_data['type'])
  text_dump += "[*][color=rgb(255, 204, 0)]Episodes:[/color] {}\n".format(anime_data['episodes'])
  text_dump += "[*][color=rgb(255, 204, 0)]Status:[/color] {}\n".format(anime_data['status'])
  text_dump += "[*][color=rgb(255, 204, 0)]Aired:[/color] {}\n".format(anime_data['aired']['string'])
  text_dump += "[*][color=rgb(255, 204, 0)]Premiered:[/color] {}\n".format(anime_data['premiered'])
  text_dump += "[*][color=rgb(255, 204, 0)]Producers:[/color] {}\n".format(get_info(anime_data, 'producers'))
  text_dump += "[*][color=rgb(255, 204, 0)]Studios:[/color] {}\n".format(get_info(anime_data, 'studios'))
  text_dump += "[*][color=rgb(255, 204, 0)]Source:[/color] {}\n".format(anime_data['source'])
  text_dump += "[*][color=rgb(255, 204, 0)]Genres:[/color] {}\n".format(get_info(anime_data, 'genres'))
  text_dump += "[*][color=rgb(255, 204, 0)]Duration:[/color] {}\n".format(anime_data['duration'])
  if mediainfo_enabled:
    text_dump += "[hr][/hr][indent][size=6][color=rgb(250, 197, 28)][b]Media Info[/b][/color][/size][/indent]\n"
    text_dump += "[spoiler='Click here to view Media Info'][code]{}[/code][/spoiler]\n".format(nfo)
  text_dump += "[hr][/hr][center][size=6][color=rgb(250, 197, 28)][b]Download Link[/b][/color][/size][/center]\n"
  if Thanks:
    if Likes != 0 and Posts == 0:
      text_dump += "[center][thanks][likes={}]{}[/likes][/thanks][/center]".format(Likes, Link)
    elif Likes == 0 and Posts != 0:
      text_dump += "[center][thanks][posts={}]{}[/posts][/thanks][/center]".format(Posts, Link)
    elif Likes != 0 and Posts != 0:
      text_dump += "[center][thanks][likes={}][posts={}]{}[/posts][/likes][/thanks][/center]".format(Likes, Posts, Link)
    else:
      text_dump += "[center][thanks]{}[/thanks][/center]".format(Link)
  else:
    if Likes != 0 and Posts == 0:
      text_dump += "[center][likes={}]{}[/likes][/center]".format(Likes, Link)
    elif Likes == 0 and Posts != 0:
      text_dump += "[center][posts={}]{}[/posts][/center]".format(Posts, Link)
    elif Likes != 0 and Posts != 0:
      text_dump += "[center][likes={}][posts={}]{}[/posts][/likes][/center]".format(Likes, Posts, Link)
    else:
      text_dump += "[center]{}[/center]".format(Link)

  return text_dump

print(get_mal_template(mal_id))
